In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
base_url = 'https://www.cava.tn/category/voitures?page=1'

detail_url = 'https://www.cava.tn'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62'
}

In [3]:
voitureslinks = []
for x in range(1,269,1):
  r = requests.get(f'https://www.cava.tn/category/voitures?page={x}.html')
  c = r.content
  soup = BeautifulSoup(c , 'html.parser')
  voitureslist = soup.find_all('app-product-item' , {"class" : "ng-star-inserted"})
  #print("le nombre est" ,len(voitureslist))
  for item in voitureslist:
    a_tag = item.find('a', class_="text-decoration-none")
    if a_tag and 'href' in a_tag.attrs:
      voitureslinks.append(a_tag['href'])
print(len(voitureslinks))    

5360


In [4]:
Les_voitures = []
for link in voitureslinks:
    voiture={}
    r = requests.get( detail_url + link , headers = headers)
    #print(r.ok)
    soup = BeautifulSoup( r.content , 'html.parser')
    try:
      titre_voiture = soup.find('h1', class_='p-title').text.strip()
      #print(titre_voiture)
      voiture["Nom Voiture"]=titre_voiture
    except:
      name = None
      print("rien")
      voiture["Nom Voiture"]=name
    try:
        price_text = soup.find('label', {'class': "px-2"}).text.strip()
        if "négociable" in price_text.lower():  # Vérifie si le texte contient "négociable"
            price = np.nan  # Remplace par NaN
            voiture["Prix"]=price
        else:
            price = price_text.replace("TND", "").strip()  # Nettoie le prix en supprimant "TND"
            voiture["Prix"]=price
    except:
        price = None
        voiture["Prix"]=price
    try:
      caracteristiques = soup.find_all('div', class_="ng-star-inserted")
      #print(len(caracteristiques))
      for item in caracteristiques:
        try:
          key = item.find('span', class_="font-p-r").text.strip()  # Label (ex: Marque de voiture)
          value = item.find('a', class_="font-p-m").text.strip()   # Valeur associée (ex: Citroen)
          voiture[key] = value
        except:
           pass
    except:
       pass
    Les_voitures.append(voiture)

In [5]:
df = pd.DataFrame(Les_voitures)
df.drop('Catégorie', axis=1, inplace=True)
df.drop('Type transaction', axis=1, inplace=True)
df

,Nom Voiture,Prix,Marque de voiture,Model,Année,Carburant,Etat,Boîte de vitesse,Couleur,Puissance fiscale,...,il y a 13 jours,il y a 4 mois,il y a 5 mois,il y a 6 mois,il y a 7 mois,il y a 8 mois,il y a 5 jours,il y a 9 mois,il y a 10 mois,il y a un an
0,Kia sportage,None,Kia,Sportage,2017,Essence,Occasion,Automatique,kaki,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Citroen GS spécial,20,Citroen,Gsa,1975,Essence,Occasion,Manuelle,blanc,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hyundai Tucson,None,Hyundai,Tucson,2022,Essence,Occasion,Manuelle,bleu,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Polo 8 1er main,1,Volkswagen,Polo,2022,Essence,Neuf,Manuelle,blanc,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BMW E24,None,Bmw,Autre,1985,Essence,Occasion,Manuelle,gris,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355,Mazda bt50 hi,65.000,Mazda,Aures,2017,Diesel,Occasion,Manuelle,marron,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5356,Fiat fiorino,21.000,Fiat,Fiorino,2018,Diesel,Occasion,Manuelle,blanc,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5357,غار الدماء,33,Peugeot,Autres,2016,Diesel,Occasion,Manuelle,blanc,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5358,C3 2003 moteur jdid,18.500,Citroen,C3,2003,Essence,Occasion,Manuelle,bleu,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#dataset to csv file 
df.to_csv('CavaCars.csv')

In [7]:
print (df.shape)

(5360, 33)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9158155e-c82c-4f23-93c0-dcd070a7ff4c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>